In [ ]:
# Imports 📥

# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

In [ ]:
class CreateDatabase:
    def __init__(self, user, password, host, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.connection = None
        self.cursor = None

    def connect(self, database=None):
        try:
            # Establish the connection to the database
            self.connection = mysql.connector.connect(
                user=self.user,
                password=self.password,
                host=self.host,
                database=database
            )
            # Create the cursor after establishing the connection
            self.cursor = self.connection.cursor()
            print(f"Connection to database '{database or 'server'}' established successfully")
        except mysql.connector.Error as err:
            print(f'Connection error: {err}')

    def create_database(self):
        try:
            # Connect without specifying a database
            self.connect(database=None)
            # Check the connection status
            if self.connection.is_connected():
                print("Temporary connection to server established for database creation.")
            else:
                print("Temporary connection to server failed.")
                return

            # Create the database using the provided name
            self.cursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.database}")
            print(f"Database '{self.database}' created or already exists.")
        except mysql.connector.Error as err:
            print(f'Error creating database: {err}')
        finally:
            # Close the temporary connection
            if self.cursor:
                self.cursor.close()
            if self.connection:
                self.connection.close()
                print("Temporary connection closed.")
        
        # Reconnect to the new database
        self.connect(database=self.database)
        # Check the connection status again
        if self.connection.is_connected():
            print(f"Connection to database '{self.database}' re-established.")
        else:
            print(f"Connection to database '{self.database}' failed.")

    def create_table(self, table_name, schema):
        try:
            # Create a table using the provided name and schema
            self.cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({schema})")
            print(f"Table '{table_name}' created or already exists.")
        except mysql.connector.Error as err:
            print(f'Error creating table: {err}')

    def insert_data(self, query, values):
        try:
            # Execute the query with the proportioned values
            self.cursor.execute(query, values)
            self.connection.commit()  # confirm the values and commit
            print(f"{self.cursor.rowcount} record(s) inserted.")

        except mysql.connector.Error as err:
            print(f'Error inserting data: {err}')
            print("Error Code:", err.errno)
            print("SQLSTATE:", err.sqlstate)
            print("Message:", err.msg)

    def bulk_insert_data(self, dataframe, table_name):
        placeholders = ", ".join(["%s"] * len(dataframe.columns))
        columns = ", ".join(dataframe.columns)
        update_clause = ", ".join([f"{col}=VALUES({col})" for col in dataframe.columns])
        query = f"""
        INSERT INTO {table_name} ({columns}) VALUES ({placeholders})
        ON DUPLICATE KEY UPDATE {update_clause}
        """

        for row in dataframe.itertuples(index=False, name=None):
            self.insert_data(query, row)

            
    def close(self):
        # Close cursor and connection
        if self.cursor:
            self.cursor.close()
        if self.connection:
            self.connection.close()
        print("Database connection closed.")

In [ ]:
# ▶️ Instantiating a class object 🐣

db_hr = CreateDatabase(user ="root", password="AlumnaAdalab", host= "127.0.0.1", database="HR_optimization") # gives parameters to the database

In [ ]:
# ▶️ Create the database#  
db_hr.create_database()

In [ ]:
# ▶️ Connection the database

db_hr.connect("HR_optimization")

In [ ]:
# ▶️ Create Table

# Define SQL statement for creating a table
table_name = ''
schema =""

db_hr.create_table(table_name, schema=)

db_hr.close()

schema1 = """CREATE TABLE IF NOT EXISTS `departments` (
  `department_id` INT NOT NULL,
  `department_name` VARCHAR(80) NULL DEFAULT NULL,
  PRIMARY KEY (`department_id`)
) ENGINE = InnoDB DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;"""

schema2= """CREATE TABLE IF NOT EXISTS `education_fields` (
  `education_field_id` INT NOT NULL AUTO_INCREMENT,
  `education_field_name` VARCHAR(80) NOT NULL,
  PRIMARY KEY (`education_field_id`),
  UNIQUE INDEX `education_field_name` (`education_field_name` ASC)
) ENGINE = InnoDB DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;"""

schema3= """CREATE TABLE IF NOT EXISTS `employees` (
  `employee_id` INT NOT NULL AUTO_INCREMENT,
  `age` INT NULL DEFAULT NULL,
  `attrition` VARCHAR(3) NULL DEFAULT NULL,
  `business_travel` VARCHAR(50) NULL DEFAULT NULL,
  `daily_rate` FLOAT NULL DEFAULT NULL,
  `department` VARCHAR(50) NULL DEFAULT NULL,
  `distance_from_home` INT NULL DEFAULT NULL,
  `education` INT NULL DEFAULT NULL,
  `education_field` VARCHAR(80) NULL DEFAULT NULL,
  `employee_number` INT NULL DEFAULT NULL,
  `environment_satisfaction` INT NULL DEFAULT NULL,
  `gender` VARCHAR(10) NULL DEFAULT NULL,
  `hourly_rate` FLOAT NULL DEFAULT NULL,
  `job_involvement` INT NULL DEFAULT NULL,
  `job_level` INT NULL DEFAULT NULL,
  `job_role` VARCHAR(50) NULL DEFAULT NULL,
  `job_satisfaction` INT NULL DEFAULT NULL,
  `marital_status` VARCHAR(30) NULL DEFAULT NULL,
  `monthly_income` FLOAT NULL DEFAULT NULL,
  `monthly_rate` FLOAT NULL DEFAULT NULL,
  `num_companies_worked` INT NULL DEFAULT NULL,
  `over_time` VARCHAR(10) NULL DEFAULT NULL,
  `percent_salary_hike` FLOAT NULL DEFAULT NULL,
  `performance_rating` INT NULL DEFAULT NULL,
  `relationship_satisfaction` INT NULL DEFAULT NULL,
  `stock_option_level` INT NULL DEFAULT NULL,
  `total_working_years` INT NULL DEFAULT NULL,
  `training_times_last_year` INT NULL DEFAULT NULL,
  `work_life_balance` INT NULL DEFAULT NULL,
  `years_at_company` INT NULL DEFAULT NULL,
  `years_since_last_promotion` INT NULL DEFAULT NULL,
  `years_with_curr_manager` INT NULL DEFAULT NULL,
  `date_birth` DATE NULL DEFAULT NULL,
  `remote_work` VARCHAR(10) NULL DEFAULT NULL,
  PRIMARY KEY (`employee_id`)
) ENGINE = InnoDB DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;"""

schema4 = """CREATE TABLE IF NOT EXISTS `job_roles` (
  `job_role_id` INT NOT NULL AUTO_INCREMENT,
  `job_role_name` VARCHAR(50) NOT NULL,
  PRIMARY KEY (`job_role_id`),
  UNIQUE INDEX `job_role_name` (`job_role_name` ASC)
) ENGINE = InnoDB DEFAULT CHARACTER SET = utf8mb4 COLLATE = utf8mb4_0900_ai_ci;
"""

In [ ]:
# ▶️ Insert Data

# Load data
hr_data = pd.read_csv("hr_data_transformed_2024-08-02_161418.csv")

job_roles = hr_data['job_role'].unique()
departments = hr_data['department'].unique()
education_fields = hr_data['education_field'].unique()

# Insert unique values into job_roles
job_role_id = 1
for role in job_roles:
    db_hr.insert_data("INSERT INTO job_roles (job_role_id, job_role_name) VALUES (%s, %s)", (job_role_id, role))
    job_role_id += 1

# Insert unique values into departments
department_id = 1
for dept in departments:
    db_hr.insert_data("INSERT INTO departments (department_id, department_name) VALUES (%s, %s)", (department_id, dept))
    department_id += 1

# Insert unique values into education_fields
education_field_id = 1
for field in education_fields:
    db_hr.insert_data("INSERT INTO education_fields (education_field_id, education_field_name) VALUES (%s, %s)", (education_field_id, field))
    education_field_id += 1



In [ ]:
hr_data = pd.read_csv("hr_data_transformed_2024-08-02_161418.csv")

In [ ]:
hr_data.rename(columns={'Unnamed: 0': 'employee_id'}, inplace=True)

# Eliminar duplicados basados en la columna 'id'
hr_data = hr_data[~hr_data.index.duplicated(keep='first')]



In [ ]:
hr_data["job_role"].unique()

In [ ]:
# Insertar datos en la tabla "employees"
db_hr.bulk_insert_data(hr_data, 'employees')
db_hr.close()